<hr>

## TABLE OF CONTENTS

1. [Packages](##1.-PACKAGES)

2. [Datasets](##2.-DATASETS)

3. [Analysis & Visualization](##3.-ANALYSIS-&-VISUALIZATION)

4. [Preprocessing & Pipeline](##4.-PREPROCESSING-&-PIPELINE)

5. [Model Prediction & Evaluation](##5.-MODEL-PREDICTION-&-EVALUATION)

6. [Model Comparison](##6.-MODEL-COMPARISON)

7. [Conclusion](##7.-CONCLUSION)

<hr>

## 1. PACKAGES

### Essential Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Machine Learning Libraries

In [2]:
# data pipeline:
from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# preprocessing:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

# model selection:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# models:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

# metrics:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, roc_curve, mean_squared_error
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

<hr>

## 2. DATASETS

### Read CSV Files

In [3]:
df_2C = pd.read_csv('datasets/orthopedic_2C.csv')
df_3C = pd.read_csv('datasets/orthopedic_3C.csv')

In [4]:
print(df_2C.shape)
print(df_3C.shape)

(310, 7)
(310, 7)


In [5]:
# 2 dfs should have the same data structure and values:
if df_2C.iloc[:, :6].equals(df_3C.iloc[:, :6]):
    # Display the first 10 rows and 6 columns of df_2c
    display(df_2C.iloc[:10, :6])

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis
0,63.027817,22.552586,39.609117,40.475232,98.672917,-0.254400
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501
5,40.250200,13.921907,25.124950,26.328293,130.327871,2.230652
6,53.432928,15.864336,37.165934,37.568592,120.567523,5.988551
7,45.366754,10.755611,29.038349,34.611142,117.270067,-10.675871
8,43.790190,13.533753,42.690814,30.256437,125.002893,13.289018
9,36.686353,5.010884,41.948751,31.675469,84.241415,0.664437


In [6]:
print(df_2C['class'].unique())
print(df_3C['class'].unique())

['Abnormal' 'Normal']
['Hernia' 'Spondylolisthesis' 'Normal']


<hr>

## 3. ANALYSIS & VISUALIZATION

### Class Distribution

#### df_2C

In [7]:
distribution_2C = pd.DataFrame({
    'Class': df_2C['class'].value_counts().index,
    'Count': df_2C['class'].value_counts().values,
    'Percentage (%)': round(df_2C['class'].value_counts(normalize=True)*100,2)
}).reindex(['Normal', 'Abnormal'])

distribution_2C = distribution_2C.reset_index(drop=True)

display(distribution_2C.style.hide(axis="index"))

Class,Count,Percentage (%)
Normal,100,32.260000
Abnormal,210,67.740000


#### df_3C

In [8]:
distribution_3C = pd.DataFrame({
    'Class': df_3C['class'].value_counts().index,
    'Count': df_3C['class'].value_counts().values,
    'Percentage (%)': round(df_3C['class'].value_counts(normalize=True)*100,2)
}).reindex(['Normal', 'Hernia', 'Spondylolisthesis'])

distribution_3C = distribution_3C.reset_index(drop=True)

display(distribution_3C.style.hide(axis="index"))

Class,Count,Percentage (%)
Normal,100,32.260000
Hernia,60,19.350000
Spondylolisthesis,150,48.390000


### Missing Values

In [9]:
display(df_2C.isnull().sum())
display(df_3C.isnull().sum())

pelvic_incidence            0
pelvic_tilt                 0
lumbar_lordosis_angle       0
sacral_slope                0
pelvic_radius               0
degree_spondylolisthesis    0
class                       0
dtype: int64

pelvic_incidence            0
pelvic_tilt                 0
lumbar_lordosis_angle       0
sacral_slope                0
pelvic_radius               0
degree_spondylolisthesis    0
class                       0
dtype: int64

<hr>

## 4. PREPROCESSING & PIPELINE

### Encoding

* OneHotEncoder for df_2C's target variable y (with 2 classes)

* LabelEncoder for df_3C's target variable y (with 3 classes)

In [10]:
# Initialize Encoder
oh_encoder = OneHotEncoder(sparse_output=False)
lb_encoder = LabelEncoder()

#### df_2C

In [11]:
# Make a copy to reserve original data
df_2C_encoded = df_2C.copy()

# Apply OneHot encoder to 'class' and transform to an array
encoded_array_2C = oh_encoder.fit_transform(df_2C_encoded[['class']])
# Create a dataframe from the encoded array
encoded_df_2C = pd.DataFrame(encoded_array_2C, columns=oh_encoder.get_feature_names_out(['class']))
# Concatenate the encoded dataframe with the original dataframe
df_2C_encoded = pd.concat([df_2C_encoded, encoded_df_2C], axis=1)
# Drop the original 'class', and 'class_Normal' column
df_2C_encoded.drop(['class','class_Normal'], axis=1, inplace=True)

# Value Counts:
display(df_2C_encoded['class_Abnormal'].value_counts().sort_index(ascending=False))

display(df_2C_encoded.head())

class_Abnormal
1.0    210
0.0    100
Name: count, dtype: int64

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class_Abnormal
0,63.027817,22.552586,39.609117,40.475232,98.672917,-0.254400,1.0
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,1.0
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,1.0
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,1.0
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,1.0


#### df_3C

In [12]:
# Make a copy to reserve original data
df_3C_encoded = df_3C.copy()

# Custom classes: 'Normal'=0, 'Hernia'=1, 'Spondylolisthesis'=2
custom_classes = ['Normal', 'Hernia', 'Spondylolisthesis']
# Force custom order
lb_encoder.classes_ = np.array(custom_classes)
# Transform using the custom order
df_3C_encoded['class'] = lb_encoder.transform(df_3C_encoded['class'])

# Value Counts:
display(df_3C_encoded['class'].value_counts().sort_index(ascending=False))

display(df_3C_encoded.head())

class
2    150
1     60
0    100
Name: count, dtype: int64

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,63.027817,22.552586,39.609117,40.475232,98.672917,-0.254400,1
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,1
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,1
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,1
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,1


### Scaling

- Since df_2C and df_3C have the exact same 6 features as proved in Section 2, the only difference is in target column. <br>
We will only scaling the features from df_2C and use it to train all models.

In [13]:
# Initate scaler:
numerical_transformer = StandardScaler()

In [14]:
numerical_cols = df_2C_encoded.columns[0:6].tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
    ],
    remainder='passthrough'
)

### Pipeline

#### Four pipelines
- DecisionTree Model
    - df_2C
    - df_3C
- Naive Bayes Model
    - df_2C
    - df_3C

In [15]:
# Classifying objects:
dt_classifier = DecisionTreeClassifier(random_state=42)
nb_classifier = GaussianNB()

##### DecisionTree model pipeline for df_2C

In [16]:
dt_pipeline_2C = Pipeline([
    ('preprocessor', preprocessor), 
    ('classifier', clone(dt_classifier))
])

##### DecisionTree model pipeline for df_3C

In [17]:
dt_pipeline_3C = Pipeline([
    ('preprocessor', preprocessor), 
    ('classifier', clone(dt_classifier))
])

##### Naive Bayes model pipeline for df_2C

In [18]:
nb_pipeline_2C = Pipeline([
    ('preprocessor', preprocessor), 
    ('classifier', clone(nb_classifier))
])

##### Naive Bayes model pipeline for df_3C

In [19]:
nb_pipeline_3C = Pipeline([
    ('preprocessor', preprocessor), 
    ('classifier', clone(nb_classifier))
])

### Splitting

In [20]:
X = df_2C_encoded[['pelvic_incidence',
 'pelvic_tilt',
 'lumbar_lordosis_angle',
 'sacral_slope',
 'pelvic_radius',
 'degree_spondylolisthesis']]

y_2C = df_2C_encoded['class_Abnormal']
y_3C = df_3C_encoded['class']

X_train_2C, X_test_2C, y_train_2C, y_test_2C = train_test_split(X, y_2C, test_size=0.2, random_state=28)
print(X_train_2C.shape, y_train_2C.shape, X_test_2C.shape, y_test_2C.shape)

X_train_3C, X_test_3C, y_train_3C, y_test_3C = train_test_split(X, y_3C, test_size=0.2, random_state=82)
print(X_train_3C.shape, y_train_3C.shape, X_test_3C.shape, y_test_3C.shape)

(248, 6) (248,) (62, 6) (62,)
(248, 6) (248,) (62, 6) (62,)


<hr>

## 5. MODEL PREDICTION & EVALUATION

### DecisionTree Model

#### df_2C

In [21]:
dt_pipeline_2C.fit(X_train_2C, y_train_2C)
y_pred_dt_2C = dt_pipeline_2C.predict(X_test_2C)
accuracy_score(y_test_2C, y_pred_dt_2C)

0.7741935483870968

#### df_3C

In [22]:
dt_pipeline_3C.fit(X_train_3C, y_train_3C)
y_pred_dt_3C = dt_pipeline_3C.predict(X_test_3C)
accuracy_score(y_test_3C, y_pred_dt_3C)

0.8387096774193549

### Gaussian Naive Bayes Model

#### df_2C

In [23]:
nb_pipeline_2C.fit(X_train_2C, y_train_2C)
y_pred_nb_2C = nb_pipeline_2C.predict(X_test_2C)
accuracy_score(y_test_2C, y_pred_nb_2C)

0.8064516129032258

#### df_3C

In [24]:
nb_pipeline_3C.fit(X_train_3C, y_train_3C)
y_pred_nb_3C = nb_pipeline_3C.predict(X_test_3C)
accuracy_score(y_test_3C, y_pred_nb_3C)

0.8225806451612904

<hr>

## 6. MODEL COMPARISON

<hr>

## 7. CONCLUSION